### Install Dependensi
Dependensi yang dibutuhkan hanya Numpy yang kita import sebagai alias np

In [ ]:
%pip install numpy

import numpy as np
'''
numpy yang sudah / akan digunakan:
1. np.linalg.det()
2. np.sqrt()
'''

Note: you may need to restart the kernel to use updated packages.


### Helper Function
Beberapa function yang akan kita gunakan beberapa kali

In [ ]:
def hitungDet(matrix): # hitung determinan langsung pakai numpy
    det = np.linalg.det(matrix)
    return det

def invalidInputMessageDeterminan(r, c): # error handling kalo ada input yang tidak sesuai
    print(f"Determinan tidak dapat ditentukan untuk matrix berukuran {r} x {c}")
    print("Hanya dapat menghitung matrix persegi (n x n)")
    print()

def tampilkanMatrix(matrix): # menampilkan matrix yang ingin ditampilkan
    for row in matrix:
        atur = ["{:.2f}".format(x) for x in row]
        print("    [" + ", ".join(atur) + "]")

### Program PPT 1 - Menyelesaikan Sistem Persamaan Linear dengan Algoritma OBE Gauss-Jordan

Pseudocode:

1. Buat matrix input
2. Buat function calls ke function utama
3. Di function utama, buat utility yang diinginkan misal print judul / header, matrix input, dan sebagainya
4. Cek dulu matrixnya kalau banyak baris tidak sama dengan banyak kolom-1 maka tampilkan pesan error (di function helper)
5. Di function utama bisa langsung buat algoritma gauss jordan:
    - Ambil pivot di diagonal yang bukan 0, jika 0 tukar baris dengan baris di bawahnya
    - Normalisasi pivot agar menjadi 1 utama (jika sudah satu utama skip)
    - Lakukan pengurangan untuk baris di bawahnya agar di bawah 1 utama semuanya 0
    - Lakukan terus hingga kolom terakhir sudah tidak ada baris di bawah 1 utama, maka lakukan lagi tetapi untuk baris di atasnya dengan algoritma yang sama
    - Tampilkan hasil matrix setiap iterasi
6. Kondisi tidak ada solusi juga terjadi ketika persamaan tidak konsisten (diagonal mendektai 0 tetapi konstanta tidak 0) dan banyak solusi (diagonal dan konstanta keduanya mendekati 0)
7. Tampilkan output hasil solusi persamaan
done!


In [3]:
def tampilkanPersamaan(matrix):
    row = len(matrix)
    col = len(matrix[0])
    variabel = col - 1

    for i in range(row):
        left = []
        for j in range(variabel):
            coef = matrix[i][j]
            left.append(f"{coef:.2f}X{j+1}")

        lhs = " + ".join(left)
        rhs = matrix[i][col-1]

        print(f"{lhs} = {rhs:.2f}")


def iterateGaussJordan(matrix):
    swap = 0        # hitung berapa kali tukar baris
    row = len(matrix)
    col = len(matrix[0])
    M = [r[:] for r in matrix]      # copy matrix ke M, aku gk mau modif matrix awal, resiko kehilangan data awal
    step = 1

    for j in range(row): # loop tiap kolom, jadi dia bisa drive gauss (sebagai index baris) sekaligus nentuin pivot. Perhatikan bahwa loop ini berhenti sampai row-1 karena kolom paling kanan tidak di-normalisasi
        pivot = M[j][j]
        if pivot == 0:
            # lakukan tukar baris
            for i in range(j+1, row): # loop tiap baris, cari pivot != 0 di bagian bawah
                if M[i][j] != 0:
                    print(f"Iterasi ke-{step}: Tukar baris ke-{j+1} dengan baris ke-{i+1}")
                    M[j], M[i] = M[i], M[j] # menukar baris
                    tampilkanMatrix(M)
                    print()
                    step += 1
                    swap += 1
                    pivot = M[j][j]
                    break
        
        if pivot == 0: continue # jaga2 kalo masih 0 kita continue ke kolom berikutnya bro

        # normalisasi pivot menjadi 1 utama
        if pivot != 1:
            print(f"Iterasi ke-{step}: Normalisasi baris ke-{j+1} / ({pivot:.2f})")
            print()
            for c in range(row): # loop tiap kolom di baris ke j (karena pivot di M[j][j])
                M[j][c] /= pivot
            tampilkanMatrix(M)
            print()
            step += 1

        # eliminasi ke semua baris di bawahnya dulu
        for r in range(j+1, row):
            if r == j or M[r][j] == 0: 
                continue 

            pengali = M[r][j]
            print(f"Iterasi ke-{step}: Baris ke-{r+1} - ({pengali:.2f}) * Baris ke-{j+1}")
            print()
            for c in range(col):
                M[r][c] -= pengali * M[j][c]

            tampilkanMatrix(M)
            print()
            step += 1

    for j in range(row-1, -1, -1):
        pivot = M[j][j]
        # skip normalisasi karena sudah dinormalisasi langsung ke eliminasi
        for r in range(j-1, -1, -1):
            if r == j or M[r][j] == 0: 
                continue 

            pengali = M[r][j]
            print(f"Iterasi ke-{step}: Baris ke-{r+1} - ({pengali:.2f}) * Baris ke-{j+1}")
            print()
            for c in range(col):
                M[r][c] -= pengali * M[j][c]

            tampilkanMatrix(M)
            print()
            step += 1
    
    print("Matrix diagonal akhir:")
    print()
    tampilkanMatrix(M)
    print()
    print("Atau persamaan:")
    tampilkanPersamaan(M)
    return M

def ambilSolusi(matrix):
    col = len(matrix[0])
    variabel = col - 1

    solusi = []

    for i in range(variabel):
        if abs(matrix[i][i]) < 1e-9: # cek apakah diagonal mendekati 0
            if abs(matrix[i][col-1]) < 1e-9: # cek apakah konstanta mendekati 0
                return None, "Gak terhingga"
            else:
                return None, "Gak onk"
        
        solusi.append(matrix[i][col-1])

    return solusi, "wes onk, aman"

def showStepGaussJordanElimination(matrix):
    print(f"===Penyelesaian SPL menggunakan OBE Gauss-Jordan===")
    print()
    print("Input matrixnya:")
    tampilkanMatrix(matrix)
    print()
    print("Atau persamaan:")
    tampilkanPersamaan(matrix)
    print()
    row = len(matrix)
    col = len(matrix[0])

    if row < col - 1: # kondisi variabel bebas
        print(f"Jumlah persamaan tidak cukup, akan ada {col - row - 1} variabel bebas")
        return

    M = iterateGaussJordan(matrix)
    solusi, status = ambilSolusi(M)

    if status == "Gak onk":
        print("Tidak ada solusi (persamaan tidak konsisten)")
    elif status == "Gak terhingga":
        print("Solusi tak hingga (sistem memiliki variabel nol semua / ada variabel bebas)")
    else:
        print()
        print("Solusinya:")
        for i, res in enumerate(solusi):
            print(f"X{i+1} = {res:.2f}")


# main driver
inputMatrix = [
    [1, 1, 1, 1],
    [1, 2, 3, 1],
    [1, 4, 9, 1],
]

showStepGaussJordanElimination(inputMatrix)

===Penyelesaian SPL menggunakan OBE Gauss-Jordan===

Input matrixnya:
     [1.00, 1.00, 1.00, 1.00]
     [1.00, 2.00, 3.00, 1.00]
     [1.00, 4.00, 9.00, 1.00]

Atau persamaan:
1.00X1 + 1.00X2 + 1.00X3 = 1.00
1.00X1 + 2.00X2 + 3.00X3 = 1.00
1.00X1 + 4.00X2 + 9.00X3 = 1.00

Iterasi ke-1: Baris ke-2 - (1.00) * Baris ke-1

     [1.00, 1.00, 1.00, 1.00]
     [0.00, 1.00, 2.00, 0.00]
     [1.00, 4.00, 9.00, 1.00]

Iterasi ke-2: Baris ke-3 - (1.00) * Baris ke-1

     [1.00, 1.00, 1.00, 1.00]
     [0.00, 1.00, 2.00, 0.00]
     [0.00, 3.00, 8.00, 0.00]

Iterasi ke-3: Baris ke-3 - (3.00) * Baris ke-2

     [1.00, 1.00, 1.00, 1.00]
     [0.00, 1.00, 2.00, 0.00]
     [0.00, 0.00, 2.00, 0.00]

Iterasi ke-4: Normalisasi baris ke-3 / (2.00)

     [1.00, 1.00, 1.00, 1.00]
     [0.00, 1.00, 2.00, 0.00]
     [0.00, 0.00, 1.00, 0.00]

Iterasi ke-5: Baris ke-2 - (2.00) * Baris ke-3

     [1.00, 1.00, 1.00, 1.00]
     [0.00, 1.00, 0.00, 0.00]
     [0.00, 0.00, 1.00, 0.00]

Iterasi ke-6: Baris ke-1 - (1.00

### Program PPT2 - Invers Matrix menggunakan OBE Gauss-Jordan

Pseudocode:

1. Buat matrix input
2. Buat function calls ke function utama
3. Di function utama, buat utility yang diinginkan misal print judul / header, matrix input, dan sebagainya
4. Cek dulu matrixnya kalau banyak baris tidak sama dengan banyak kolom maka tampilkan pesan error (di function helper)
5. Buat matrix identity.
5. Di function utama bisa langsung buat algoritma gauss jordan:
    - Ambil pivot di diagonal yang bukan 0, jika 0 tukar baris dengan baris di bawahnya
    - Normalisasi pivot agar menjadi 1 utama (jika sudah satu utama skip)
    - Lakukan pengurangan untuk baris di bawahnya agar di bawah 1 utama semuanya 0
    - Lakukan terus hingga kolom terakhir sudah tidak ada baris di bawah 1 utama, maka lakukan lagi tetapi untuk baris di atasnya dengan algoritma yang sama
    - Hal yang sama dilakukan di matrix identity (lakukan bersamaan dengan input matrix)
    - Tampilkan hasil matrix augmented setiap iterasi
6. Tampilkan output hasil invers matrix
done!

In [4]:
def usingGaussJordanInversion(M, I):
    row = len(M)
    col = len(M[0])
    step = 1
    swap = 0

    print("Proses OBE")
    print()

    for j in range(row):
        pivot = M[j][j]

        if pivot == 0:
            for i in range(j+1, row):
                if M[i][j] != 0:
                    print(f"Iterasi ke-{step}: Tukar baris ke-{j+1} dengan baris ke-{i+1}")
                    M[i], M[j] = M[j], M[i]
                    I[i], I[j] = I[j], I[i]
                    print("Matrix menjadi: ")
                    aug = []
                    for baris in range(row):
                        aug.append(M[baris] + I[baris])

                    tampilkanMatrix(aug)
                    print()
            
        if pivot == 0: continue # jaga2 kalo masih 0 kita continue ke kolom berikutnya bro

        # normalisasi pivot menjadi 1 utama
        if pivot != 1:
            print(f"Iterasi ke-{step}: Normalisasi baris ke-{j+1} / ({pivot:.2f})")
            print()
            for c in range(row): # loop tiap kolom di baris ke j (karena pivot di M[j][j])
                M[j][c] /= pivot
                I[j][c] /= pivot

            aug = []
            for baris in range(row):
                aug.append(M[baris] + I[baris])

            tampilkanMatrix(aug)
            print()
            step += 1

        # eliminasi ke semua baris di bawahnya dulu
        for r in range(j+1, row):
            if r == j or M[r][j] == 0: 
                continue 

            pengali = M[r][j]
            print(f"Iterasi ke-{step}: Baris ke-{r+1} - ({pengali:.2f}) * Baris ke-{j+1}")
            print()
            for c in range(col):
                M[r][c] -= pengali * M[j][c]
                I[r][c] -= pengali * I[j][c]

            aug = []
            for baris in range(row):
                aug.append(M[baris] + I[baris])

            tampilkanMatrix(aug)
            print()
            step += 1

    for j in range(row-1, -1, -1):
        pivot = M[j][j]
        # skip normalisasi karena sudah dinormalisasi langsung ke eliminasi
        for r in range(j-1, -1, -1):
            if r == j or M[r][j] == 0: 
                continue 

            pengali = M[r][j]
            print(f"Iterasi ke-{step}: Baris ke-{r+1} - ({pengali:.2f}) * Baris ke-{j+1}")
            print()
            for c in range(col):
                M[r][c] -= pengali * M[j][c]
                I[r][c] -= pengali * I[j][c]

            aug = []
            for baris in range(row):
                aug.append(M[baris] + I[baris])

            tampilkanMatrix(aug)
            print()
            step += 1
    
    return M, I, swap




def showStepGaussJordanInversion(matrix):
    print(f"===Invers Matrix Menggunakan Gauss Jordan")
    print()
    print("Input matrixnya:")
    tampilkanMatrix(matrix)
    print()

    row = len(matrix)
    col = len(matrix[0])

    M = [r[:] for r in matrix]
    I = [] # buat matrix identitas

    if row != col: 
        print("Tidak dapat ditentukan inversnya")
        invalidInputMessageDeterminan(row, col)
        return
    else:
        for i in range(row):
            I.append([])
            for j in range(col):
                if j == i:
                    I[i].append(1)
                else:
                    I[i].append(0)
    
    M , I, swap = usingGaussJordanInversion(M, I)

    print("Matrix awal:")
    tampilkanMatrix(matrix)
    print()
    print("Menjadi identitas:")
    tampilkanMatrix(M)
    print()
    print("Matrix inversnya:")
    tampilkanMatrix(I)
    print()
    print(f"Telah dilakukan {swap} kali tukar baris")


# main driver
inputMatrix = [
    [1, 1, 1],
    [1, 2, 3],
    [1, 4, 9],
]

showStepGaussJordanInversion(inputMatrix)

===Invers Matrix Menggunakan Gauss Jordan

Input matrixnya:
     [1.00, 1.00, 1.00]
     [1.00, 2.00, 3.00]
     [1.00, 4.00, 9.00]

Proses OBE

Iterasi ke-1: Baris ke-2 - (1.00) * Baris ke-1

     [1.00, 1.00, 1.00, 1.00, 0.00, 0.00]
     [0.00, 1.00, 2.00, -1.00, 1.00, 0.00]
     [1.00, 4.00, 9.00, 0.00, 0.00, 1.00]

Iterasi ke-2: Baris ke-3 - (1.00) * Baris ke-1

     [1.00, 1.00, 1.00, 1.00, 0.00, 0.00]
     [0.00, 1.00, 2.00, -1.00, 1.00, 0.00]
     [0.00, 3.00, 8.00, -1.00, 0.00, 1.00]

Iterasi ke-3: Baris ke-3 - (3.00) * Baris ke-2

     [1.00, 1.00, 1.00, 1.00, 0.00, 0.00]
     [0.00, 1.00, 2.00, -1.00, 1.00, 0.00]
     [0.00, 0.00, 2.00, 2.00, -3.00, 1.00]

Iterasi ke-4: Normalisasi baris ke-3 / (2.00)

     [1.00, 1.00, 1.00, 1.00, 0.00, 0.00]
     [0.00, 1.00, 2.00, -1.00, 1.00, 0.00]
     [0.00, 0.00, 1.00, 1.00, -1.50, 0.50]

Iterasi ke-5: Baris ke-2 - (2.00) * Baris ke-3

     [1.00, 1.00, 1.00, 1.00, 0.00, 0.00]
     [0.00, 1.00, 0.00, -3.00, 4.00, -1.00]
     [0.00, 0.0

### Program PPT3 - Determinan Matrix Menggunakan OBE Gauss

Pseudocode:

1. Buat matrix input
2. Buat function calls ke function utama
3. Di function utama, buat utility yang diinginkan misal print judul / header, matrix input, dan sebagainya
4. Cek dulu matrixnya kalau banyak baris tidak sama dengan banyak kolom maka tampilkan pesan error (di function helper)
5. Di function utama bisa langsung buat algoritma gauss jordan:
    - Ambil pivot di diagonal yang bukan 0, jika 0 tukar baris dengan baris di bawahnya
    - Normalisasi pivot agar menjadi 1 utama (jika sudah satu utama skip)
    - Lakukan pengurangan untuk baris di bawahnya agar di bawah 1 utama semuanya 0
    - Lakukan terus hingga kolom terakhir sudah tidak ada baris di bawah 1 utama, maka lakukan lagi tetapi untuk baris di atasnya dengan algoritma yang sama
6. Tampilkan output hasil determinan.
done!

In [5]:
def gaussAlgorithm(matrix):
    swap = 0        # hitung berapa kali tukar baris
    row = len(matrix)
    M = [r[:] for r in matrix]      # copy matrix ke M, aku gk mau modif matrix awal, resiko kehilangan data awal
    step = 1

    for j in range(row): # loop tiap kolom, jadi dia bisa drive gauss (sebagai index baris) sekaligus nentuin pivot
        pivot = M[j][j]
        if pivot == 0:
            # lakukan tukar baris
            for i in range(j+1, row): # loop tiap baris, cari pivot != 0 di bagian bawah
                if M[i][j] != 0:
                    print(f"Iterasi ke-{step}: Tukar baris ke-{j+1} dengan baris ke-{i+1}")
                    print()
                    M[j], M[i] = M[i], M[j] # menukar baris
                    tampilkanMatrix(M)
                    print()
                    step += 1
                    swap += 1
                    pivot = M[j][j]
                    break
        
        # sudah ketemu pivot kita eliminasi
        for i in range(j+1, row): # loop tiap baris ke bawah, kek di bagian tukar baris
            if M[i][j] == 0: continue

            pengali = M[i][j] / pivot
            print(f"Iterasi ke-{step}: Baris ke-{i+1} = Baris ke-{i+1} - ({pengali:.2f}) * Baris ke-{j+1}")
            print()

            for c in range(row): # loop tiap kolom buat OBE. Kita pakai row juga soalnya row == col (bisa dihitung determinannya)
                M[i][c] -= pengali * M[j][c] # j adalah index baris si pivot. Karena pivotnya selalu M[j][j]

            tampilkanMatrix(M)
            print()
            step += 1

    return M, swap


def showStepGaussDeterminant(matrix):
    print(f"===Determinan matrix dengan Algoritma Gauss===")
    print()
    print("Matrix Input:")

    tampilkanMatrix(matrix)
    print()
    row = len(matrix)
    col = len(matrix[0])

    if row == col:
        print(f"Iterasi OBE gauss untuk menentukan determinan matrix {row} x {col}")
        print()
        M, swap = gaussAlgorithm(matrix)
        det = 1
        elemenDiagonal = []

        for i in range(row):
            elemenDiagonal.append(f"({M[i][i]})")
            det *= M[i][i]

        tandaSwap = pow(-1, swap)
        det *= tandaSwap
        print("Hasil determinan matrix input:")
        tampilkanMatrix(matrix)
        print()

        print("Menjadi matrix segitiga atas:")
        tampilkanMatrix(M)
        print()

        print("Determinan = " + " * ".join(elemenDiagonal))
        print(f"Hasil akhir: {det:.2f}, karena ada {swap} kali tukar baris")
        print()

    else:
        invalidInputMessageDeterminan(row, col)
                    
                    
# main driver
inputMatrix = [
    [2, 1, -1, 8],
    [-3, -1, 2, -11],
    [-2, 1, 2, -3],
    [2, 4, 1, 3]
]

showStepGaussDeterminant(inputMatrix)

===Determinan matrix dengan Algoritma Gauss===

Matrix Input:
     [2.00, 1.00, -1.00, 8.00]
     [-3.00, -1.00, 2.00, -11.00]
     [-2.00, 1.00, 2.00, -3.00]
     [2.00, 4.00, 1.00, 3.00]

Iterasi OBE gauss untuk menentukan determinan matrix 4 x 4

Iterasi ke-1: Baris ke-2 = Baris ke-2 - (-1.50) * Baris ke-1

     [2.00, 1.00, -1.00, 8.00]
     [0.00, 0.50, 0.50, 1.00]
     [-2.00, 1.00, 2.00, -3.00]
     [2.00, 4.00, 1.00, 3.00]

Iterasi ke-2: Baris ke-3 = Baris ke-3 - (-1.00) * Baris ke-1

     [2.00, 1.00, -1.00, 8.00]
     [0.00, 0.50, 0.50, 1.00]
     [0.00, 2.00, 1.00, 5.00]
     [2.00, 4.00, 1.00, 3.00]

Iterasi ke-3: Baris ke-4 = Baris ke-4 - (1.00) * Baris ke-1

     [2.00, 1.00, -1.00, 8.00]
     [0.00, 0.50, 0.50, 1.00]
     [0.00, 2.00, 1.00, 5.00]
     [0.00, 3.00, 2.00, -5.00]

Iterasi ke-4: Baris ke-3 = Baris ke-3 - (4.00) * Baris ke-2

     [2.00, 1.00, -1.00, 8.00]
     [0.00, 0.50, 0.50, 1.00]
     [0.00, 0.00, -1.00, 1.00]
     [0.00, 3.00, 2.00, -5.00]

Iterasi ke-

Program PPT4-Determinan dengan Ekspansi Kofaktor

Pseudocode:

1. Buat matrix input
2. Buat function calls ke function utama
3. Di function utama, buat utility yang diinginkan misal print judul / header, matrix input, dan sebagainya
4. Cek dulu matrixnya kalau banyak baris tidak sama dengan banyak kolom maka tampilkan pesan error (di function helper)
5. Di function utama bisa langsung buat algoritma ekspansi kofaktor (disini menggunakan baris pertama):
    - ambil elemen baris 1 kolom 1
    - Tentukan minor dan determinannya (menggunakan numpy)
    - kalikan elemen dengan tanda pada kofaktor dan dikalikan dengan determinan kofaktor
    - tambahkan hasil tadi ke determinan total
    - ulangi sampai baris 1 kolom terakhir
6. Tampilkan output hasil determinannya
done!

In [6]:
def minorMatrix(matrix, row, col):
    minor = []

    for i in range(len(matrix)):
        if i == row:
            continue

        newBaris = []

        for j in range(len(matrix[i])):
            if j == col:
                continue

            newBaris.append(matrix[i][j])
        
        minor.append(newBaris)

    return minor

def showStep(matrix):
    print("=== Determinan Menggunakan Ekspansi Kofaktor Baris kesatu ===")
    print("Matrix input:")
    tampilkanMatrix(matrix)
    print()
    row = len(matrix)
    col = len(matrix[0])

    if row == col:
        print(f"Determinan dari matrix input ukuran {row} x {col}:")
        det = 0
        kalkulasi = []
        for j in range (row):
            tanda = pow(-1, j)
            elemen = matrix[0][j]
            print(f"Elemen ke-{j+1}: {elemen}")
            if tanda > 0: 
                print("Tanda: Positif")
            else:
                print("Tanda: Negatif")
            
            print("Minornya:")
            minor = minorMatrix(matrix, 0, j)
            tampilkanMatrix(minor)
            print(f"Determinan minornya: {hitungDet(minor):.2f}")
            print()

            kalkulasi.append(f"{tanda} * {elemen} * det(Minor[0][{j}])")
            det += tanda * elemen * hitungDet(minor)

        print("Hasil perhitungan lengkap determinannya:")
        print("Determinan = " + " + ".join(kalkulasi))
        print()
        print(f"Hasil akhir determinan = {det:.2f}")
        print()
    else:
        invalidInputMessageDeterminan(row, col)


# main driver
inputMatrix = [
    [2, 1, -1, 8],
    [-3, -1, 2, -11],
    [-2, 1, 2, -3],
    [2, 4, 1, 3]
]

showStep(inputMatrix)


=== Determinan Menggunakan Ekspansi Kofaktor Baris kesatu ===
Matrix input:
     [2.00, 1.00, -1.00, 8.00]
     [-3.00, -1.00, 2.00, -11.00]
     [-2.00, 1.00, 2.00, -3.00]
     [2.00, 4.00, 1.00, 3.00]

Determinan dari matrix input ukuran 4 x 4:
Elemen ke-1: 2
Tanda: Positif
Minornya:
     [-1.00, 2.00, -11.00]
     [1.00, 2.00, -3.00]
     [4.00, 1.00, 3.00]
Determinan minornya: 38.00

Elemen ke-2: 1
Tanda: Negatif
Minornya:
     [-3.00, 2.00, -11.00]
     [-2.00, 2.00, -3.00]
     [2.00, 1.00, 3.00]
Determinan minornya: 39.00

Elemen ke-3: -1
Tanda: Positif
Minornya:
     [-3.00, -1.00, -11.00]
     [-2.00, 1.00, -3.00]
     [2.00, 4.00, 3.00]
Determinan minornya: 65.00

Elemen ke-4: 8
Tanda: Negatif
Minornya:
     [-3.00, -1.00, 2.00]
     [-2.00, 1.00, 2.00]
     [2.00, 4.00, 1.00]
Determinan minornya: -5.00

Hasil perhitungan lengkap determinannya:
Determinan = 1 * 2 * det(Minor[0][0]) + -1 * 1 * det(Minor[0][1]) + 1 * -1 * det(Minor[0][2]) + -1 * 8 * det(Minor[0][3])

Hasil akhi